<a href="https://colab.research.google.com/github/bird0401/Instance_level_recognition/blob/main/02_resnet_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning==0.7.1

     |████████████████████████████████| 6.0 MB 11.1 MB/s 
     |████████████████████████████████| 829 kB 58.9 MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-0.7.1-py3-none-any.whl size=145329 sha256=b3118d5906390f37ef8692937a6af4e991f8f4f7da1fb31140de8ae27fbabf4a
  Stored in directory: /root/.cache/pip/wheels/a5/c0/6c/ed64904da20814878f410c520ae61c062c6d7e93bf5c27dcd4
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=9b251b16ba978147ebc6bc3765c76fd3f03a0d5044371ee9f070e441c4e57fd2
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built pytorch-lightning future
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [2]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer

import torch,torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.models import resnet18
import torch.utils.data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split

from collections import defaultdict
import os
import random
from tqdm import tqdm
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset

In [29]:
categories=["dog","car_02","uspresident","uspresident_02","tower","tower_02"]
category=categories[1]+"_100"

In [30]:
# !cp /content/drive/MyDrive/instance_level_recognition/dog_2.tar.gz .
!cp /content/drive/MyDrive/instance_level_recognition/imgs_{category}.tar.gz .

In [47]:
!tar -zxvf imgs_{category}.tar.gz

imgs_car_02_100/
imgs_car_02_100/Q914069_018.jpg
imgs_car_02_100/Q370524_092.jpg
imgs_car_02_100/Q926873_058.jpg
imgs_car_02_100/Q1511429_022.jpg
imgs_car_02_100/Q748949_055.jpg
imgs_car_02_100/Q23830984_012.jpg
imgs_car_02_100/Q23830984_015.jpg
imgs_car_02_100/Q370524_076.JPG
imgs_car_02_100/Q214229_042.JPG
imgs_car_02_100/Q3476815_001.jpg
imgs_car_02_100/Q826094_051.jpg
imgs_car_02_100/Q2003307_093.tif
imgs_car_02_100/Q23498122_002.jpg
imgs_car_02_100/Q1801831_009.JPG
imgs_car_02_100/Q2003307_130.JPG
imgs_car_02_100/Q826094_116.jpg
imgs_car_02_100/Q370524_129.jpg
imgs_car_02_100/Q2003307_137.jpg
imgs_car_02_100/Q1511429_016.jpg
imgs_car_02_100/Q225600_001.JPG
imgs_car_02_100/Q22673219_015.jpg
imgs_car_02_100/Q594990_023.jpg
imgs_car_02_100/Q2003307_126.jpg
imgs_car_02_100/Q1368464_016.jpg
imgs_car_02_100/Q497884_004.jpg
imgs_car_02_100/Q830375_091.jpg
imgs_car_02_100/Q914069_002.JPG
imgs_car_02_100/Q2443038_030.JPG
imgs_car_02_100/Q830375_094.jpg
imgs_car_02_100/Q2003307_052.jpg
imgs

In [48]:
!ls imgs_{category}|wc -l

2875


In [49]:
!rm -rf /content/{category}.tar.gz

In [50]:
# # 224×224で正規化した画像に変換

class ImageTranform():
  def __init__(self):
    self.data_transform=transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
  def __call__(self,img):
    return self.data_transform(img)

In [51]:
# transform=ImageTranform()
# img=Image.open("images/Q17504_001.jpg")
# img=transform(img)
# # img=img.numpy().transpose((1,2,0)) # imshow用に次元入れ替え
# img=np.clip(img,0,1)
# plt.imshow(img)
# plt.show()
# print(img.shape)
# print(img.dtype)

In [52]:
# 実行後のファイルはすでに保存してあるのでコメントアウト

# # image_size=32 #試し用
# image_size=224 

# image_shape=(image_size,image_size,3)

# ファイルの入出力用にglobをインポート
import glob

image_dir = "imgs_"+category
paths=glob.glob(image_dir+"/*")

## 準備

# q_files = defaultdict(list)
imgs,qids=[],[]
transform=ImageTranform()

for path in tqdm(paths):
  try:
    # img=np.array(Image.open(path).convert("RGB").resize((image_size,image_size)))
    img=Image.open(path)
    img=transform(img)
    img=img.numpy()
    img=np.clip(img,0,1)
  except Exception as e:
    print(e)
  else:
    ## キーと画像のリストを保存
    filename=path.split("/")[1]
    qid = filename.split("_")[0][1:]
    qids.append(qid)
    imgs.append(img)
    ## キーから画像を取得できるようにする
    # q_files[qid].append(img)


  0%|          | 12/2875 [00:03<10:09,  4.69it/s]

output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]



  3%|▎         | 77/2875 [00:15<08:50,  5.27it/s]

output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]



  4%|▍         | 113/2875 [00:20<06:22,  7.22it/s]

...


 96%|█████████▋| 2772/2875 [07:55<00:22,  4.49it/s]

output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]



 98%|█████████▊| 2825/2875 [08:02<00:03, 12.66it/s]

output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]



100%|██████████| 2875/2875 [08:10<00:00,  5.86it/s]


In [55]:
# #  pickleで保存
with open('qids_'+category+'.binaryfile','wb') as f:
  pickle.dump(qids,f)

with open('imgs_'+category+'.binaryfile','wb') as f:
  pickle.dump(imgs,f)

In [56]:
# # 保存したファイルをdriveにコピー
!cp /content/{'qids_'+category+'.binaryfile'} /content/drive/MyDrive/instance_level_recognition
!cp /content/{'imgs_'+category+'.binaryfile'} /content/drive/MyDrive/instance_level_recognition

In [14]:
# # 保存したファイルをdriveから取ってくる
# !cp /content/drive/MyDrive/instance_level_recognition/{'qids_'+category+'.binaryfile'} .
# # !cp /content/drive/MyDrive/instance_level_recognition/imgs.binaryfile .

In [15]:
# # pickleからロード
# with open('/content/qids_'+category+'.binaryfile','rb') as f:
#   qids=pickle.load(f)
# # with open('/content/imgs.binaryfile','rb') as f:
# #   imgs=pickle.load(f)

In [57]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(qids)
y_labels=le.transform(qids)

In [58]:
len(qids)

2801

In [59]:
len(y_labels)

2801

In [60]:
n_classes=len(set(y_labels.tolist()))

In [61]:
imgs=torch.tensor(imgs,dtype=torch.float32)
y_labels=torch.from_numpy(y_labels.astype(np.uint8)).clone()

# X_train, X_valid, X_test = X_train/255.0, X_valid/255.0, X_test/255.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [62]:
#  pickleで保存
with open('y_labels_'+category+'.binaryfile','wb') as f:
  pickle.dump(y_labels,f)

with open('imgs_'+category+'.binaryfile','wb') as f:
  pickle.dump(imgs,f)

In [63]:
# 保存したファイルをdriveにコピー
!cp /content/{'y_labels_'+category+'.binaryfile'} /content/drive/MyDrive/instance_level_recognition
!cp /content/{'imgs_'+category+'.binaryfile'} /content/drive/MyDrive/instance_level_recognition

In [21]:
# # 保存したファイルをdriveから取ってくる
# !cp /content/drive/MyDrive/instance_level_recognition/{'y_labels_'+category+'.binaryfile'} .
# !cp /content/drive/MyDrive/instance_level_recognition/'imgs_'+category+'.binaryfile' .

In [22]:
# # pickleからロード
# with open('y_labels_'+category+'.binaryfile','rb') as f:
#   y_labels=pickle.load(f)
# with open('imgs_'+category+'.binaryfile','rb') as f:
#   imgs=pickle.load(f)

In [64]:
# 統合
dataset=torch.utils.data.TensorDataset(imgs,y_labels)
dataset

In [65]:
n_train = int(len(dataset) * 0.6)
n_val = int((len(dataset) - n_train) * 0.5)
n_test = len(dataset) - n_train - n_val
n_train, n_val, n_test

(1680, 560, 561)

In [66]:
torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])
# len(X_train),len(X_valid),len(X_test)

# Resnet18

In [67]:
class TrainNet(pl.LightningModule):
  def train_dataloader(self):
    return torch.utils.data.DataLoader(train,self.batch_size,shuffle=True)
  def training_step(self,batch,batch_nb):
    x,t=batch
    y=self.forward(x)
    loss=self.lossfun(y,t)
    results={"loss":loss}
    return results

class ValidationNet(pl.LightningModule):
  def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)
  def validation_step(self,batch,batch_nb):
    x,t=batch
    y=self.forward(x)
    loss=self.lossfun(y,t)
    y_label=torch.argmax(y,dim=1)
    acc=torch.sum(t==y_label)/len(t)
    results={"val_loss":loss,"val_acc":acc}
    return results
  def validation_end(self,outputs):
    avg_loss=torch.stack([x["val_loss"] for x in outputs]).mean()
    avg_acc=torch.stack([x["val_acc"] for x in outputs]).mean()
    results={"val_loss":avg_loss,"val_acc":avg_acc}
    return results

class TestNet(pl.LightningModule):
    def test_dataloader(self):
        return torch.utils.data.DataLoader(test, self.batch_size)

    def test_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'test_loss': loss, 'test_acc': acc}
        return results

    def test_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        results = {'test_loss': avg_loss, 'test_acc': avg_acc}
        return results

In [68]:
resnet=resnet18(pretrained=True)

class Net(TrainNet,ValidationNet, TestNet):
  def __init__(self,batch_size=256):
    super().__init__()
    self.batch_size=batch_size
    self.conv=resnet
    self.fc=nn.Linear(1000,n_classes)
    for param in self.conv.parameters():
      param.requires_grad=False
      
  def lossfun(self, y, t):
      return F.cross_entropy(y, t)

  def configure_optimizers(self):
      return torch.optim.SGD(self.parameters(), lr=0.01)

  def forward(self, x):
      x = self.conv(x)
      x = self.fc(x)
      return x

In [71]:
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(0)

net=Net()
trainer=Trainer(gpus=1,max_epochs=30,batch_size=1024)
trainer.fit(net)

Validation sanity check:   0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

1

# Results

In [72]:
trainer.test()
trainer.callback_metrics

Testing:   0%|          | 0/3 [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
TEST RESULTS
{}
----------------------------------------------------------------------------------------------------


{'epoch': 29,
 'loss': 1.2475230693817139,
 'test_acc': 0.38026151061058044,
 'test_loss': 2.5646307468414307,
 'val_acc': 0.4088541865348816,
 'val_loss': 2.64506196975708}